In [1]:
# modular exponentiation: a^m = 1 mode n
# if a and n are coprime, then there is at least one integer m that satisfies that equation where m = phi(n)
# where phi(n) is the Euler's totient function

# order finding problem:
# find the smallest positive integer r such that a^r = 1 mod n
# r is called the order of a modulo n
# if a and n are coprime, then the order of a modulo n divides phi(n)
# phi(n) % r = 0

In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
from qiskit_aer import AerSimulator
from qiskit.compiler import transpile
from fractions import Fraction
import numpy as np

In [3]:
# i do not understand this part yet
# how on earth does this work?
# IBM Qiskit textbook says that this is a controlled modular exponentiation


# controlled modular exponentiation: a^power mod 15
def c_amod15(a, power):
    qreg = QuantumRegister(4)
    qc = QuantumCircuit(qreg, name=f"{a}^{power} mod 15")
    for _ in range(power):
        if a in [2, 13]:
            qc.swap(2, 3)
            qc.swap(1, 2)
            qc.swap(0, 1)
        if a in [7, 8]:
            qc.swap(0, 1)
            qc.swap(1, 2)
            qc.swap(2, 3)
        if a in [4, 11]:
            qc.swap(1, 3)
            qc.swap(0, 2)
        if a in [7, 11, 13]:
            qc.x(qreg)
    return qc.to_gate().control(1)

In [4]:
def phase_estimation(a, N):
    n = 4
    ancilla = 1
    aqreg = QuantumRegister(ancilla)
    sqreg = QuantumRegister(n)
    creg = ClassicalRegister(ancilla)
    qc = QuantumCircuit(aqreg, sqreg, creg)

    qc.h(aqreg)
    qc.x(sqreg[0])
    for i in range(ancilla):
        qc.append(c_amod15(a, 2**i), [aqreg[i]] + [*sqreg])
    qc.append(QFT(ancilla).inverse(), aqreg)

    qc.measure(aqreg, creg)

    return qc, ancilla

In [5]:
def order_finding(qc, ancilla, N):
    simulator = AerSimulator()
    isa_circuit = transpile(qc, simulator)
    result = simulator.run(isa_circuit).result()
    counts = result.get_counts()
    highest_probability_outcome = max(counts, key=counts.get)
    phase = int(highest_probability_outcome, 2) / 2**ancilla  # theta = s / r
    r = Fraction(phase).limit_denominator(N).denominator
    return r

In [6]:
# this method succeeds in finding a factor of N with probability at least 1/2
# providing N is odd and not a prime power
def factorization():
    N = 15
    Zn_star = [i for i in range(2, N - 1) if np.gcd(i, N) == 1]
    i = 0
    while True:
        i += 1
        a = np.random.choice(Zn_star)
        print(f"ATTEMPT {i}: a = {a}")
        d = np.gcd(a, N)
        if d >= 2:
            return d, N // d
        qc, ancilla = phase_estimation(a, N)
        r = order_finding(qc, ancilla, N)
        if r % 2 == 0:
            d = np.gcd(a ** (r // 2) - 1, N)
            if d >= 2:
                print(f"order of {a} mod {N} is {r}")
                return d, N // d
        print("FAILED")

In [7]:
factors = factorization()
print(f"factors of {15}: {factors}")

ATTEMPT 1: a = 8
FAILED
ATTEMPT 2: a = 13
order of 13 mod 15 is 2
factors of 15: (3, 5)
